# Data Preprocessing

## Import Libraries and Dataset

We first would import the necessary libraries we need at the moment, we will import others as when they are needed

In [68]:
import pandas as pd # for data manipulation
import joblib # for saving tools

Import dataset

In [69]:
lagos = pd.read_csv("clean/lagos.csv")

In [70]:
# show first 5 rows
lagos.head()

,Title,Location,Currency,Price,Serviced,Newly Built,Furnished,Bedrooms,Bathrooms,Toilets,Neighborhood
0,Shops,Allen Avenue Roundabout Allen Avenue Ikeja Lagos,₦,"750,000/sqm",1,0,0,0 beds,0 baths,0 Toilets,Allen Avenue
1,Newly Built 5 Bedrooms Detached Triplex,"Eso Close, Off Oduduwa Crescent, Gra Ikeja, La...",₦,"280,000,000/year",0,1,0,5 beds,5 baths,6 Toilets,GRA
2,1800m2 Land,Off Adedayo Banjo Street Opebi Ikeja Lagos,₦,"220,000,000",0,0,0,0 beds,0 baths,0 Toilets,Opebi
3,5 Bedroom Fully Detached Duplex,Magodo Ikeja Lagos,₦,"300,000,000",0,1,0,5 beds,6 baths,6 Toilets,Other Ikeja
4,Luxury Built 4bedroom Fully Detached Duplex,Ikeja Lagos,₦,"295,000,000",1,1,0,4 beds,5 baths,5 Toilets,Other Ikeja


In [71]:
# shoe basic information of the dataset
lagos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66383 entries, 0 to 66382
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         66382 non-null  object
 1   Location      66383 non-null  object
 2   Currency      66383 non-null  object
 3   Price         66383 non-null  object
 4   Serviced      66383 non-null  int64 
 5   Newly Built   66383 non-null  int64 
 6   Furnished     66383 non-null  int64 
 7   Bedrooms      66383 non-null  object
 8   Bathrooms     66383 non-null  object
 9   Toilets       66383 non-null  object
 10  Neighborhood  66383 non-null  object
dtypes: int64(3), object(8)
memory usage: 5.6+ MB


We can see from the data above we have 10 features as our Target Variable is **Price**

In [72]:
lagos.columns

Index(['Title', 'Location', 'Currency', 'Price', 'Serviced', 'Newly Built',
       'Furnished', 'Bedrooms', 'Bathrooms', 'Toilets', 'Neighborhood'],
      dtype='object')

## Feature Engineering

#### Extracting Type of Property from Title

Since we are selling homes, let's parse out the three main types of Property Types from the title column: land, duplex and apartment

In [73]:
def house_type(serie):
    final = []
    
    # Extract property name from title
    for title in list(serie):
        # convert all titles to lowercase trings
        title = str(title).lower()
        if "land" in title:
            final.append("land")
        elif ("duplex" in title) or ("massionette" in title):
            final.append("duplex")
        else:
            final.append("apartment")
    return final

In [74]:
lagos["Property Type"] = house_type(lagos["Title"])

In [75]:
lagos[["Title","Property Type"]].sample(5)

,Title,Property Type
14799,Water Front Land,land
49906,Just Built 5 Bedroom Fully Detached Smart Home...,duplex
40611,4 Bedroom Fully Detached Duplexes,duplex
62736,Urban City Tastefully Finished 3 Bedroom Apart...,apartment
49415,5 Bedroom Detached Duplex With A Bq,duplex


We only need duplex properties, so it will be set

In [76]:
lagos = lagos[lagos["Property Type"] == "duplex"].copy()

#### Extracting Type of Duplex from Title

We can see different types of homes like detached duplex, semi detached etc, lets extract that

In [77]:
lagos["Title"].value_counts()

4 Bedroom Semi Detached Duplex                                                        1479
5 Bedroom Detached Duplex                                                             1088
4 Bedroom Terrace Duplex                                                               767
5 Bedroom Fully Detached Duplex                                                        761
4 Bedroom Detached Duplex                                                              467
                                                                                      ... 
New Automated Unconventional 4bedroom Terrace Duplex With Bq Swimming Pool And Gym       1
5 Bedroom Contemporary Fully Detached Duplex + Bq                                        1
Distressed 4bedrooms Fully Detached Duplex                                               1
New 4bedeoom Semi Detached Duplex With Bq                                                1
7 Bedroom Duplex + 2 Room Bq                                                             1

In [78]:
# Seperate Detached Duplex Houses from Semi Detached houses
lagos.loc[
    (lagos["Title"] == "Detached Duplex") | (lagos["Title"] == "5 Bedroom Detached Duplex") |
    (lagos["Title"] == "4 Bedroom Detached Duplex") | (lagos["Title"] == "6 Bedroom Detached Duplex"),
    "Title"
] = "ONLY DETACHED DUPLEX"

This is a function to differentiate whether a home is detached, semi-detached, terraced or massionette

In [79]:
def type_house(col):
    final = []
    
    for value in col:
        if ("ONLY DETACHED DUPLEX" in value.upper()) or ('bedroom duplex' in value.lower()):
            final.append("Detached Duplex")
        elif ('Fully Detached' in value) or ('bedroom detached duplex' in value.lower()):
            final.append("Detached Duplex")
        elif ('bedrooms duplex' in value.lower()) or ('bedrooms detached' in value.lower()):
            final.append("Detached Duplex")
        elif ('bed room' in value.lower()) or ('luxury' in value.lower()) or ('contemporary' in value.lower()):
            final.append("Detached Duplex")
        elif "semi" in value.lower():
            final.append("Semi Detached Duplex")
        elif "terra" in value.lower():
            final.append("Terraced Duplex")
        else:
            final.append("Massionette")
            
    return final

In [80]:
# Applying the function
lagos["Type"] = type_house(lagos["Title"])

In [81]:
# Checking count of type of duplexes
lagos["Type"].value_counts()

Detached Duplex         19216
Semi Detached Duplex     7542
Terraced Duplex          5425
Massionette               870
Name: Type, dtype: int64

We won't be needing the Title column again

In [82]:
lagos.drop("Title", axis=1, inplace=True)

We dont also need houses that are massionette

In [83]:
mask = lagos.loc[lagos['Type'] == 'Massionette'].index
lagos.drop(mask, inplace=True)

#### Cleaning and Casting of the Price column

It seems our price column is a string, as it contains commas and also includes the price period of the properties

Let's sort that out

In [84]:
# A sample of price values
lagos["Price"].sample(5)

61574    160,000,000
16608    360,000,000
41560    220,000,000
13528    160,000,000
1818     550,000,000
Name: Price, dtype: object

Function to differentiate whether a price is fixed or per sqm, per year etc

In [85]:
# Remove the last item from the price
def price_split(serie):
    final = []
    for value in serie:
        # if value has a price period, extract it else let it's price period be fixed
        try:
            final.append(value.split("/")[1])
        except:
            final.append("fixed")
    return final

In [86]:
# applying function
lagos["Price Period"] = price_split(lagos["Price"])

In [87]:
# checking samples
lagos[["Price", "Price Period"]].sample(5)

,Price,Price Period
29675,"80,000,000",fixed
20937,"120,000,000",fixed
37436,"200,000,000",fixed
35045,"85,000,000",fixed
52041,"55,000,000",fixed


Now since we have parsed out the price period, let's remove the commas from them and convert them to numbers

In [88]:
def comma_remove(feature):
    final = []
    for sample in feature:
        # Get the prices
        sample = sample.split("/")[0]
        
        # Remove the commas
        if "," in sample:
            final.append(sample.replace(",",""))
        else:
            final.append(sample)
    
    # Convert them to numbers
    int_final = [float(item) for item in final]
    return int_final


In [89]:
# applying functions
lagos["Price"] = comma_remove(lagos["Price"])

In [90]:
lagos[["Price", "Price Period"]].sample(5)

,Price,Price Period
24896,200000000.0,fixed
60537,65000000.0,fixed
8489,165000000.0,day
36768,95000000.0,fixed
21626,40000000.0,fixed


Extracting houses only with fixed prices

In [91]:
lagos = lagos[lagos['Price Period'] == 'fixed']

We can now perform numerical methods on the Price column now

Minimum price

In [92]:
lagos["Price"].min()

0.0

Average price

In [93]:
lagos["Price"].median()

100000000.0

Max Price

We can see that some of our other columns meant to be numbers are actually strings, so we would have take care of that

In [94]:
lagos["Price"].max()

140000000000000.0

#### Cleaning and Casting of the Bedrooms, Bathrooms and Toilets Features

In [95]:
lagos[["Bedrooms","Bathrooms","Toilets"]].sample(5)

,Bedrooms,Bathrooms,Toilets
51689,5 beds,5 baths,6 Toilets
42270,5 beds,6 baths,6 Toilets
64604,4 beds,0 baths,0 Toilets
22554,4 beds,0 baths,0 Toilets
6092,5 beds,5 baths,6 Toilets


Function to clean and cast bathroom, bedroom and toilets features

In [96]:
def remove_bed_strip(serie):
    final = []
    # If value doesn't have a number, replace it with a zero,
    # else replace it with the number it contains
    for value in serie:
        if value.split("beds")[0].strip() == "":
            final.append(int("0"))
        else:
            final.append(int(value.split("beds")[0].strip()))
    return final

def remove_toilet_strip(serie):
    final = []
    for value in serie:
        if value.split("Toilets")[0].strip() == "":
            final.append(int("0"))
        else:
            final.append(int(value.split("Toilets")[0].strip()))
    return final 

def remove_bath_strip(serie):
    final = []
    for value in serie:
        if value.split("baths")[0].strip() == "":
            final.append(int("0"))
        else:
            final.append(int(value.split("baths")[0].strip()))
    return final 

Applying functions to three features

In [97]:
lagos["Bedrooms"] = remove_bed_strip(lagos["Bedrooms"])

In [98]:
lagos["Bathrooms"] = remove_bath_strip(lagos["Bathrooms"])

In [99]:
lagos["Toilets"] = remove_toilet_strip(lagos["Toilets"])

In [100]:
lagos[["Bedrooms","Bathrooms","Toilets"]].sample(5)

,Bedrooms,Bathrooms,Toilets
41941,0,0,0
29680,4,5,5
31607,5,5,6
37139,4,5,6
33042,6,6,8


#### Extracting City of property

Also we need to parse out the necessary neighbourhoods from the Location column

In [101]:
# Remove location from data
def location_parser(serie):
    # Replace all Spaces with a comma in the values
    location = [location.replace(" ", ",") for location in serie]
    
    # Split the values by commas, and get the second to the last item, also replace items that
    # contain Island with Victoria Island
    area = [area.split(",")[-2].replace("Island", "Victoria Island") for area in location]
    
    return area

Applying function

In [102]:
lagos["City"] = location_parser(lagos["Location"])

In [103]:
lagos["City"].sample(5)

58156    Lekki
43664    Lekki
19175    Lekki
38006    Lekki
36304    Lekki
Name: City, dtype: object

We wont be needing the Location feature again since City has been extracted

In [104]:
lagos.drop("Location", axis=1, inplace=True)

In [105]:
lagos.sample(10)

,Currency,Price,Serviced,Newly Built,Furnished,Bedrooms,Bathrooms,Toilets,Neighborhood,Property Type,Type,Price Period,City
11205,₦,750000000.0,0,1,0,0,0,0,Old Ikoyi,duplex,Detached Duplex,fixed,Ikoyi
7980,₦,180000000.0,0,1,0,4,4,5,Parkview,duplex,Terraced Duplex,fixed,Ikoyi
58192,₦,150000000.0,0,0,0,5,0,0,Other Lekki,duplex,Detached Duplex,fixed,Lekki
63354,₦,175000000.0,0,1,0,4,4,5,Oniru,duplex,Detached Duplex,fixed,Victoria Island
23285,₦,78000000.0,0,0,0,0,0,0,Other Lekki,duplex,Semi Detached Duplex,fixed,Lekki
35800,₦,60000000.0,0,1,0,4,5,5,Other Lekki,duplex,Semi Detached Duplex,fixed,Lekki
21281,₦,110000000.0,0,1,0,5,5,6,Lekki Phase 2,duplex,Detached Duplex,fixed,Lekki
47082,₦,38000000.0,0,0,0,0,0,0,Other Lekki,duplex,Terraced Duplex,fixed,Lekki
917,₦,180000000.0,0,0,1,5,5,5,Allen Avenue,duplex,Detached Duplex,fixed,Ikeja
52650,₦,110000000.0,0,1,1,5,5,6,Ikota,duplex,Detached Duplex,fixed,Lekki


#### Handling Missing Data

After Analysis of the data it was observed that most properties' bathrooms and bedrooms had the same values , while toilets were an increment of the bedrooms. For example a property containing 4 bedrooms will have 4 bathrooms and 5 toilets. Lets turn this idea into a function

In [106]:
# Imputing for bathrooms and bedrooms
def bed_bath(cols):
    bedroom = cols[0]
    bathroom = cols[1]
    
    if bathroom == 0:

        if bedroom == 4:
            return 4

        elif bedroom == 5:
            return 5
        
        elif bedroom == 6:
            return 6

        else:
            return 0
    else:
        return bathroom

In [107]:
# Importing for toilets
def bed_toilet(cols):
    bedroom = cols[0]
    toilet = cols[1]
    
    if toilet == 0:

        if bedroom == 4:
            return 5

        elif bedroom == 5:
            return 6

        elif bedroom == 6:
            return 7
        
        else:
            return 0
    else:
        return toilet

Applying the functions

In [108]:
lagos['Bedrooms'] = lagos[['Bathrooms','Bedrooms']].apply(bed_bath,axis=1)
lagos['Bathrooms'] = lagos[['Bedrooms','Bathrooms']].apply(bed_bath,axis=1)
lagos['Toilets'] = lagos[['Bedrooms','Toilets']].apply(bed_toilet,axis=1)

#### Handling duplicate values

Counting Number of duplicate values

In [109]:
lagos.duplicated().value_counts()[1]

16357

Removing duplicate values

In [110]:
lagos.drop_duplicates(inplace=True)

#### Removing Inconsistent rows and features

Extracting properties that contain only naira curriencies

In [111]:
lagos = lagos.loc[lagos["Currency"] != "$"].copy()

Removing rows that containing only 0 bedrooms, 0 bathrooms and 0 toilets

In [112]:
mask = lagos.loc[(lagos['Bedrooms'] == 0) & (lagos['Bedrooms'] == 0) & (lagos['Toilets'] == 0)].index
lagos.drop(mask, inplace=True)

Removing Inconsistent Features

In [113]:
lagos.drop(['Currency', 'Property Type', 'Price Period'], axis=1, inplace=True)

##### Price Cap


The average Nigerian in 2022 will not normally buy a property less than 10 million naira and higher than 500 million naira, this rows are redundant and will be removed

In [114]:
mask = lagos.loc[(lagos["Price"] < 1e7) | (lagos["Price"] > 5e8)].index
lagos.drop(mask, inplace=True)

In [115]:
lagos = lagos.sample(frac=1).copy()

Saving preprocessed data for EDA

In [116]:
lagos.to_csv('clean/eda.csv', index=False)

## Data Preprocessing

#### Data Split

Splitting our data before data preprocessing to avoid data leakage

In [117]:
# Spliting to extract 20% test data
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(lagos, lagos["Neighborhood"]):
    train_set = lagos.iloc[train_index]
    test_set = lagos.iloc[test_index]

In [118]:
# Splitting to extract 20% validation data
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(train_set, train_set["Neighborhood"]):
    train = train_set.iloc[train_index]
    validation_set = train_set.iloc[test_index]

In [119]:
lagos = train.drop("Price", axis=1)
price = train["Price"]

Let's seperate our numerical columns and our categorical ones

In [120]:
categorical_col = [column for column in lagos.columns if lagos[column].dtype == "O"]
numerical_col = ['Bathrooms', 'Bedrooms', 'Toilets']

In [121]:
lagos[categorical_col].sample(4)

,Neighborhood,Type,City
47624,Other Lekki,Detached Duplex,Lekki
23384,Other Lekki,Detached Duplex,Lekki
18327,Lekki Phase 1,Detached Duplex,Lekki
63357,Other Victoria Island,Detached Duplex,Victoria Island


In [122]:
lagos[numerical_col].sample(4)

,Bathrooms,Bedrooms,Toilets
55730,4,4,5
44918,4,4,4
25239,5,5,5
38132,5,4,5


#### Handling Missing Data

Our dataset does not have missing values, but we still need to set a foundation, in case new data cotains some missing values

In [123]:
# import library for missing data 
from sklearn.impute import SimpleImputer

Load the imputer and replacing missing categorical variables with their mode

In [124]:
categorical_imputer = SimpleImputer(strategy="most_frequent")
categorical_imputer.fit(lagos[categorical_col])
lagos[categorical_col] = categorical_imputer.transform(lagos[categorical_col])

Saving Imputer for future use in model deployment

In [125]:
joblib.dump(categorical_imputer, "tools/imputer_joblib")

['tools/imputer_joblib']

#### Feature Scaling

Scaling for all our features to be in the same range of -3 to 3

In [126]:
# import library for scaling
from sklearn.preprocessing import StandardScaler

In [127]:
# load the scaler and scale the values
scaler = StandardScaler()
scaler.fit(lagos[numerical_col])
lagos[numerical_col] = scaler.transform(lagos[numerical_col])

Saving Scaler for future use in model deployment

In [128]:
joblib.dump(scaler, "tools/scaler_joblib")

['tools/scaler_joblib']

#### Feature Encoding

We need to perform OneHotEncoding on some of the norminal categorical variables (variables with no order e.g. Nigeria, Congo, Ghana)

In [129]:
# import library for encoding
from feature_engine.encoding import OneHotEncoder

In [130]:
encoder = OneHotEncoder()
encoder.fit(lagos)
lagos = encoder.transform(lagos)

Saving Encoder for future use in model deployment

In [131]:
joblib.dump(encoder, "tools/encoder_joblib")

['tools/encoder_joblib']

In [132]:
lagos

,Serviced,Newly Built,Furnished,Bedrooms,Bathrooms,Toilets,Neighborhood_Opebi,Neighborhood_Other Lekki,Neighborhood_Adeniyi Jones,Neighborhood_Lekki Phase 1,...,Neighborhood_Airport Road,Neighborhood_Awolowo Way,Neighborhood_Agidingbi,Type_Semi Detached Duplex,Type_Detached Duplex,Type_Terraced Duplex,City_Ikeja,City_Lekki,City_Ikoyi,City_Victoria Island
3619,0,0,0,-0.481408,-0.582738,-0.170592,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
35085,0,0,0,-0.481408,-0.582738,0.736067,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
4148,0,0,0,4.529648,3.233198,2.549384,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
61009,0,0,0,-0.481408,-0.582738,-1.077250,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
44805,0,1,0,0.771356,1.325230,-0.170592,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54238,1,1,0,0.771356,-0.582738,-0.170592,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
25839,0,1,0,0.771356,0.371246,0.736067,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
61773,0,0,0,-0.481408,-0.582738,-0.170592,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
22561,0,1,0,0.771356,1.325230,0.736067,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


Save preprocessed data for Model Building

In [133]:
lagos = pd.concat([lagos, price], axis=1)

In [134]:
lagos.to_csv('clean/train.csv', index=False)
test_set.to_csv('clean/test.csv', index=False)
validation_set.to_csv('clean/validation.csv', index=False)